In [7]:
import json
import spacy

nlp = spacy.blank("fr")

# Load the JSON data
with open('annotations.json') as f:
    data = json.loads(f.read())


# Convert annotations to spaCy format
spacy_data = []
for annotation in data['annotations']:
    text = annotation[0]
    entities = annotation[1]['entities']
    entity_tuples = [(start, end, label) for start, end, label in entities]
    spacy_data.append({'text': text, 'entities': entity_tuples})
    # print(text , entities)

# Print the converted spaCy data
print (spacy_data)


[{'text': "J'ai été infecté par un virus récemment.", 'entities': [(24, 39, 'B34.9')]}, {'text': "Je suis actuellement traité ou j'ai été récemment traité avec un antibiotique pris par la bouche pour une infection de l'oreille ou une otite.", 'entities': [(65, 128, 'H6690'), (136, 141, 'H6690')]}, {'text': "Je suis infecté par le virus de l'immunodéficience humaine ou je suis séropositif.", 'entities': [(32, 58, 'HIV'), (70, 81, 'HIV')]}, {'text': "J'ai déjà eu une péricardite.", 'entities': [(17, 28, 'I30')]}, {'text': 'Moi ou un membre de ma famille avons déjà eu le croup.', 'entities': [(45, 53, 'J05.0')]}, {'text': "J'ai déjà fait de l'eau sur les poumons.", 'entities': [(18, 39, 'J81')]}, {'text': 'Je souffre de pancréatite chronique.', 'entities': [(14, 35, 'K86.1')]}, {'text': "J'ai une mauvaise alimentation.", 'entities': [(9, 30, 'MAUV_ALIMENT')]}, {'text': 'Je suis dialysé.', 'entities': [(8, 15, 'Z99.2')]}, {'text': "J'ai des ganglions enflés ou douloureux.", 'entities': [(9

In [8]:
import random
from spacy.training.example import Example

# Add NER component to the pipeline
ner = nlp.add_pipe("ner")

# Define the labels for NER
ner.add_label("SYMPTOMS")

# Convert training data to spaCy Example objects
examples = []

for train_example in spacy_data:
    text = train_example["text"]
    entities = train_example["entities"]
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, {"entities": entities})
    examples.append(example)

# Disable other pipeline components except NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    # Initialize the optimizer
    optimizer = nlp.initialize()

    # Training loop
    n_epochs = 100  # Number of training epochs

    for epoch in range(n_epochs):
        random.shuffle(examples)
        losses = {}
        nlp.update(examples, sgd=optimizer, losses=losses)
        print(f"Epoch: {epoch+1} Loss: {losses}")

# Save the trained model
nlp.to_disk("new_new_model")

Epoch: 1 Loss: {'ner': 5651.268764972687}
Epoch: 2 Loss: {'ner': 5650.381702065468}
Epoch: 3 Loss: {'ner': 5648.788575708866}
Epoch: 4 Loss: {'ner': 5645.891540884972}
Epoch: 5 Loss: {'ner': 5639.979927182198}
Epoch: 6 Loss: {'ner': 5625.948770284653}
Epoch: 7 Loss: {'ner': 5587.818479955196}
Epoch: 8 Loss: {'ner': 5463.465309381485}
Epoch: 9 Loss: {'ner': 4986.657637894154}
Epoch: 10 Loss: {'ner': 3325.0427681803703}
Epoch: 11 Loss: {'ner': 1260.4781767614186}
Epoch: 12 Loss: {'ner': 1040.5764670015778}
Epoch: 13 Loss: {'ner': 1099.9134739863248}
Epoch: 14 Loss: {'ner': 1110.9848747683818}
Epoch: 15 Loss: {'ner': 1097.8596176555254}
Epoch: 16 Loss: {'ner': 1058.2806946545566}
Epoch: 17 Loss: {'ner': 981.9137755211386}
Epoch: 18 Loss: {'ner': 891.2669522356391}
Epoch: 19 Loss: {'ner': 817.2140438364004}
Epoch: 20 Loss: {'ner': 790.6805435204296}
Epoch: 21 Loss: {'ner': 830.4210352611262}
Epoch: 22 Loss: {'ner': 1077.4686343874782}
Epoch: 23 Loss: {'ner': 1240.6034139590338}
Epoch: 24 L

In [18]:
#testing the model
my_model = spacy.load("new_model")
tester = ["J'ai une douleur qui s'améliore lorsque je penche mon corps vers l'avant." ,"J'ai un cancer actif." ,"J'ai une diminution de l'appétit ou des boissons."]

for test in tester:
    doc = my_model(test)
    for ent in doc.ents:
        start = ent.start_char
        end = ent.end_char
        label = ent.label_
        text = ent.text

        print(f"Entity: {text}, Label: {label}, Start: {start}, End: {end}")

Entity: douleur qui s'améliore lorsque je penche mon corps vers l'avant., Label: BW_BENDING, Start: 9, End: 73
Entity: cancer actif, Label: C00-D48, Start: 8, End: 20
Entity: diminution de l'appétit ou des boissons., Label: BOIRE_PED, Start: 9, End: 49
